# Inhibitor Diffusion and Germination - Slow Release

## Numerical experiments - Week 45/2024

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

## Prerequisite libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numba import jit, cuda, float32

## 1. General assumptions

Medium and conidium sizes:
- The diameter of a spore is assumed to be $5 \mu m$.
- The experiments are performed on a three-dimensional cube lattice, in which a spore is approximated by a sphere of diameter $5 \mu \textrm{m}$ (radius $R=2.5\mu \textrm{m}$) in the middle of the lattice.
- The lattice of size $L\times L\times L$ with $L=1\textrm{mm}$ is discretised in segments of size $\delta x = 1 \mu \textrm{m}$. This resuls in 1000 subdivisions along each dimension.
- The spherical spore therefore contains 81 lattice sites and has a volume $V=\frac{4}{3}\pi R^3\approx 65.45\mu \textrm{m}^3=6.525\times 10^{-8}\textrm{mm}^3$

In [2]:
def invoke_smart_kernel(size, threads_per_block=(8, 8, 8)):
    """
    Invoke a kernel with the appropriate number of blocks and threads per block.
    """
    blocks_per_grid = [(size + (tpb - 1)) // tpb for tpb in threads_per_block]
    return tuple(blocks_per_grid), tuple(threads_per_block)

@cuda.jit
def change_c_init_GPU(c_lattice, c_spore_init, spore_center_coords, spore_rad, dx):
    """
    Parallel initialisation of the lattice with the spore concentration.
    """
    i, j, k = cuda.grid(3)
    if i < c_lattice.shape[0] and j < c_lattice.shape[1] and k < c_lattice.shape[2]:
        if (i - spore_center_coords[0])**2 + (j - spore_center_coords[1])**2 + (k - spore_center_coords[2])**2 <= (spore_rad/dx)**2:
            c_lattice[i, j, k] = c_spore_init

def define_spore_region(spore_rad, c_spore_init, spore_center_coords, c_lattice, dx):
    """
    Define the spore region in the lattice and initialise inhibitor concentrations in it.
    inputs:
        spore_rad: int, radius of the spore region
        c_spore_init: float, initial concentration of the spore region
        spore_center_coords: tuple or ndarray, coordinates of the spore region
        c_lattice: np.array, lattice of inhibitor concentrations
        dx: float, spatial step size
    """

    spore_center_coords = np.array(spore_center_coords)

    assert spore_center_coords.shape == (3,), "spore_center_coords must be a 3D coordinate"
    assert c_lattice.ndim == 3, "c_lattice must be a 3D lattice"

    d_c_lattice = cuda.to_device(c_lattice)

    grid_dims, block_dims = invoke_smart_kernel(c_lattice.shape[0])

    change_c_init_GPU[grid_dims, block_dims](d_c_lattice, c_spore_init, spore_center_coords, spore_rad, dx)

    c_lattice = d_c_lattice.copy_to_host()

    interior_sites = np.sum(c_lattice)

    print("Interior sites:", interior_sites)

    return c_lattice

# Example usage
W, H, L = 1000, 1000, 1000
c_lattice = np.zeros((W, H, L), dtype=np.float32)
spore_rad = 0.0025
c_spore_init = 1.0
spore_idx = (W // 2, H // 2, L // 2)
dx = 0.001

c_lattice = define_spore_region(spore_rad, c_spore_init, spore_idx, c_lattice, dx)

# X, Y, Z = np.meshgrid(np.arange(W), np.arange(H), np.arange(L))

# fig = go.Figure(data=go.Isosurface(
#     x=X.flatten(),
#     y=Y.flatten(),
#     z=Z.flatten(),
#     value=c_lattice.flatten(),
#     isomin=c_spore_init-0.01,
#     isomax=c_spore_init+0.01,
#     caps=dict(x_show=False, y_show=False)
#     ))
# fig.show()

c:\Users\mihbo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numba\cuda\cudadrv\devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Interior sites: 81.0


## 2. Mathematical framework

The discretisation scheme is as in the previous experiment, but also accounts for the third dimension, yielding the update scheme:

$$
c_{i,j,k}^{n+1} = \frac{D\delta{t}}{\delta{x^2}}(c_{i+1,j,k}^{n}+c_{i-1,j,k}^{n}+c_{i,j+1,k}^{n}+c_{i,j-1,k}^{n}+c_{i,j,k+1}^{n}+c_{i,j,k-1}^{n}-6c_{i,j,k}^{n}) + c_{i,j,k}^{n}
$$

In [ ]:
def invoke_smart_kernel(size, threads_per_block=(16, 16, 16)):
    """
    Invoke a kernel with the appropriate number of blocks and threads per block.
    """
    blocks_per_grid = [(size + (tpb - 1)) // tpb for tpb in threads_per_block]
    return tuple(blocks_per_grid), tuple(threads_per_block)


@cuda.jit()
def update_GPU(c_old, c_new, Ddtdx2):
    """
    Update the concentration of a lattice point based on the time-dependent diffusion equation with a periodic boundary.
    inputs:
        c_old (numpy.ndarray) - the current state of the lattice;
        c_new (numpy.ndarray) - the next state of the lattice;
        Ddtdx2 (float) - the update factor.
    """
    i, j, k = cuda.grid(3)

    center = c_old[i, j, k]
    bottom = c_old[(i + 1) % c_old.shape[0], j, k]
    top = c_old[(i - 1) % c_old.shape[0], j, k]
    left = c_old[i, (j - 1) % c_old.shape[1], k]
    right = c_old[i, (j + 1) % c_old.shape[1], k]
    back = c_old[i, j, (k - 1) % c_old.shape[2]]
    forward = c_old[i, j, (k + 1) % c_old.shape[2]]

    c_new[i, j] = Ddtdx2 * (bottom + top + left + right + back + forward - 6 * center) + center


def diffusion_time_dependent_GPU(c_init, t_max, D=1.0, dt=0.001, dx=0.0005, n_save_frames=100, c_thresh=None):
    """
    Compute the evolution of a square lattice of concentration scalars
    based on the time-dependent diffusion equation.
    inputs:
        c_init (numpy.ndarray) - the initial state of the lattice;
        t_max (int) - a maximum number of iterations;
        D (float) - the diffusion constant; defaults to 1;
        dt (float) - timestep; defaults to 0.001;
        dx (float) - spatial increment; defaults to 0.005;
        n_save_frames (int) - determines the number of frames to save during the simulation; detaults to 100;
        c_thresh (float) - a threshold value for the concentration; defaults to None.
    outputs:
        u_evolotion (numpy.ndarray) - the states of the lattice at all moments in time.
    """

    assert c_init.ndim == 3, 'input array must be 3-dimensional'
    assert all([c_init.shape[i-1] == c_init.shape[i] for i in range(3)]), 'lattice must have equal size along each dimension'

    # Determine number of lattice rows/columns
    N = c_init.shape[0]

    # Save update factor
    Ddtdx2 = D * dt / (dx ** 2)

    if  Ddtdx2 > 0.5:
        print("Warning: inappropriate scaling of dx and dt, may result in an unstable simulation.")

    # Determine number of frames
    n_frames = int(np.floor(t_max / dt)) + 1
    print(f"Simulation running for {n_frames} steps on a lattice of size {np.array(c_init.shape) * dx} mm.")

    # Array for storing lattice states
    # c_evolution = np.zeros((n_save_frames + 1, N, N))
    # times = np.zeros(n_save_frames + 1)
    # save_interval = np.floor(n_frames / n_save_frames)
    # save_ct = 0
    time_thresh = None

    # Initialise lattice states
    print("Saving first lattice to device.")
    c_A_gpu = cuda.to_device(c_init)
    print("Saving second lattice to device.")
    c_B_gpu = cuda.device_array_like(c_init)
    print("Lattices saved to device.")
    c_grids = [c_A_gpu, c_B_gpu]

    kernel_blocks, kernel_threads = invoke_smart_kernel(N)

    for t in range(n_frames):

        print(f"Frame {t} of {n_frames}", end="\r")

        c_curr = c_grids[t%2]
        c_next = c_grids[(t+1)%2]
        update_GPU[kernel_blocks, kernel_threads](c_curr, c_next, Ddtdx2)

        # Save time if threshold is reached
        if c_thresh is not None and time_thresh is None:
            if cuda.reduce(c_grids[(t+1)%2].ravel(), cuda.maximum) < c_thresh:
                time_thresh = t * dt

        # Save frame
        # if t % save_interval == 0:
        #     c_evolution[save_ct] = c_grids[(t+1)%2].copy_to_host()
        #     times[save_ct] = t * dt
        #     save_ct += 1

    c_final = c_grids[(n_frames+1)%2].copy_to_host()

    return c_final, time_thresh

## 3. Experimental setup

In [ ]:
# # Run simulation
c_thresh = 0.1 * c_spore_init
D = 0.0006
t_max = 1
c_final, time_thresh = diffusion_time_dependent_GPU(c_lattice, t_max=t_max, D=D, dx=dx, dt=0.0001, n_save_frames=4, c_thresh=c_thresh)

# Check with analytical solution
# spore_vol = 1.25e-7
# print(f"Total concentration at final step: {np.sum(c_evolution[-1])}")
# c_analytical = diffusion_time_dependent_analytical(c_spore_init, D, t_max, spore_vol)
# print(f"Numerical solution at spore for t_max={t_max}: {c_evolution[-1, spore_idx[0], spore_idx[1]]}")
# print(f"Analytical solution at spore for t_max={t_max}: {c_analytical}")

Simulation running for 10001 steps on a lattice of size [1. 1. 1.] mm.
Saving first lattice to device.
Saving second lattice to device.
Lattices saved to device.


c:\Users\mihbo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
